In [1]:
import mne 
import numpy as np
import os

In [4]:
raw_data_path = r'D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif'
raw = mne.io.read_raw_fif(raw_data_path, preload=True)

Opening raw data file D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif...
    Range : 0 ... 162659 =      0.000 ...   650.636 secs
Ready.
Reading 0 ... 162659  =      0.000 ...   650.636 secs...


In [5]:
events = mne.find_events(raw, stim_channel='STI 014')


Finding events on: STI 014
1421 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 71 72 73 74 90]


In [6]:
sfreq = raw.info['sfreq']
prim_codes = [71, 72, 73, 74]
face_codes = list(range(1, 21))

for code in prim_codes:
    prim_samples = events[events[:, 2] == code, 0]
    print(f"\nCode {code}:")
    for s in prim_samples[:5]:  # just look at first 5 trials per condition
        # find nearest face event (any 1–20)
        diffs = events[np.isin(events[:, 2], face_codes), 0] - s
        nearest_idx = np.argmin(np.abs(diffs))
        delay_ms = diffs[nearest_idx] / sfreq * 1000
        print(f"  nearest face = {events[np.isin(events[:, 2], face_codes)][nearest_idx, 2]}, delay = {delay_ms:.1f} ms")



Code 71:
  nearest face = 3, delay = 432.0 ms
  nearest face = 13, delay = 488.0 ms
  nearest face = 17, delay = 484.0 ms
  nearest face = 11, delay = 496.0 ms
  nearest face = 9, delay = 380.0 ms

Code 72:
  nearest face = 13, delay = 540.0 ms
  nearest face = 19, delay = 480.0 ms
  nearest face = 1, delay = 384.0 ms
  nearest face = 19, delay = 496.0 ms
  nearest face = 13, delay = 424.0 ms

Code 73:
  nearest face = 10, delay = 480.0 ms
  nearest face = 16, delay = 496.0 ms
  nearest face = 14, delay = 436.0 ms
  nearest face = 10, delay = 524.0 ms
  nearest face = 14, delay = 504.0 ms

Code 74:
  nearest face = 16, delay = 436.0 ms
  nearest face = 10, delay = 464.0 ms
  nearest face = 14, delay = 488.0 ms
  nearest face = 12, delay = 472.0 ms
  nearest face = 2, delay = 512.0 ms


In [7]:
new_trigger_offset = 100  # 71->171, 72->172, etc.
new_events = []

for prim_code in [71, 72, 73, 74]:
    prim_samples = events[events[:,2] == prim_code, 0]
    face_samples = events[np.isin(events[:,2], range(1,21)), 0]
    face_codes = events[np.isin(events[:,2], range(1,21)), 2]

    for s in prim_samples:
        # compute delays to all face triggers
        diffs = (face_samples - s).astype(float)  # <- convert to float
        diffs[diffs < 0] = np.inf  # ignore faces before the prime
        if np.all(diffs == np.inf):
            continue  # no future face found
        nearest_idx = np.argmin(diffs)
        face_sample = face_samples[nearest_idx]
        new_code = prim_code + new_trigger_offset
        new_events.append([face_sample, 0, new_code])


In [8]:
out_suffix = '_face_onset_copy'

In [9]:
all_events = np.vstack([events, np.array(new_events, int)])
all_events = all_events[np.argsort(all_events[:, 0])]

# save events
out_dir = os.path.dirname(raw_data_path)
base = os.path.splitext(os.path.basename(raw_data_path))[0]
out_events_fname = os.path.join(out_dir, base + out_suffix + '-events.fif')
mne.write_events(out_events_fname, all_events)
print(f"Saved new events to: {out_events_fname}")

Saved new events to: D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw_face_onset_copy-events.fif


C:\Users\LSA\AppData\Local\Temp\ipykernel_24504\373600613.py:8: RuntimeWarning: This filename (D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw_face_onset_copy-events.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(out_events_fname, all_events)
